In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from matplotlib import dates
from mpl_toolkits.axes_grid1 import ImageGrid, make_axes_locatable, host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.radarmodule as radar
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from scipy import interpolate
from metpy.plots import StationPlot
import metpy.calc as mpcalc
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
from scipy.signal import medfilt2d
import pyart
import cartopy.crs as ccrs
from IPython.display import HTML
# Use pysteps for advection correction and temporal interpolation
from pysteps import io, motion, rcparams
from scipy.ndimage import map_coordinates
%matplotlib inline
# %matplotlib notebook
import warnings;
warnings.filterwarnings('ignore')

In [ ]:
# Function definitions
def roundPartial(value, resolution, decimals=4):
    return np.around(np.round(value / resolution) * resolution, decimals=decimals)

def mtokm(val,pos):
    """Convert m to km for formatting axes tick labels"""
    val=val/1000.0
    return '%i' % val

def interpolate_all(gridded_radar, tinterp_intv, base_field_name='reflectivity_masked', use_base_field=False):
    # Get list of intervals in seconds between subsequent radar times
    tdiffs = gridded_radar['time_seconds'].diff(dim='time')
    
    # This list will hold all the time-interpolated grids (xarray Datasets). 
    # Can later be concatenated into a new xarray Dataset containing all times
    gridded_radar_interp_list = []
    
    # Grab first time from full dataset and restore singular time dimension
    first_time_ds = gridded_radar.isel(time=0)
    first_time_ds = first_time_ds.expand_dims(dim='time')

    gridded_radar_interp_list.append(first_time_ds)
    
#     tbgn = first_time_ds.coords['time_seconds'].values.item()  # Need to get scalar value, not 0-d
#                                                                # numpy array
    
    # Loop through the gridded_radar times, perform advection correction/interpolation between successive times
    # and add each to the list, making sure the time coordinate is consistent
    # new_time = tbgn
    for i, tdiff in enumerate(tdiffs.values):
        gridded_radar_interp_sublist = advection_correction_ds(gridded_radar.isel(time=slice(i, i+2)), 
                                                               tdiff, tinterp_intv, 
                                                               base_field_name=base_field_name,
                                                               use_base_field=use_base_field)
        for t, gridded_radar_interp in enumerate(gridded_radar_interp_sublist):
#             new_time = new_time + tinterp_intv
#             new_ds = first_time_ds.copy()
#             new_ds[:] = gridded_radar_interp
#             new_ds.coords['time'] = new_ds['time'] + np.timedelta64(int(new_time), 's')
#             new_ds.coords['time_seconds'] = new_time
            gridded_radar_interp_list.append(gridded_radar_interp)
    
    return gridded_radar_interp_list


def advection_correction_ds(radar_ds, tintv_obs, tintv, base_field_name='reflectivity_masked', 
                            use_base_field=False, method="LK"):
    # Evaluate advection
    oflow_method = motion.get_method(method)
    fd_kwargs = {"buffer_mask": 10}  # avoid edge effects

    # Decide whether to use a single optical flow field for all fields in the Dataset (using the "base" field)
    # or to use separate ones for each field
    base_field = radar_ds[base_field_name]
    if use_base_field:
        oflow_field = oflow_method(base_field, fd_kwargs=fd_kwargs)
        oflow_field_list = [oflow_field] * len(radar_ds.items())
    else:
        oflow_field_list = []
        for field_name, field_da in radar_ds.items():
            oflow_field = oflow_method(field_da, fd_kwargs=fd_kwargs)
            oflow_field_list.append(oflow_field)
            
    # Perform temporal interpolation on all variables in Dataset using the flow field derived from either 
    # the "base" field (by default, reflectivity), or each field individually
    
    tbgn = base_field[0].coords['time_seconds'].values.item()   # Need to get scalar value, not 0-d
                                                                # numpy array
    print(tbgn)
    print(tintv)
    radar_ds_list = []
    x, y = np.meshgrid(
        np.arange(base_field[0].shape[1], dtype=float), np.arange(base_field[0].shape[0], dtype=float),
    )
    
    new_time = tbgn
    for i in np.arange(tintv, tintv_obs + tintv, tintv):
        new_time = new_time + tintv
        field_interp_list = []
        for field_da, oflow_field in zip(radar_ds.values(), oflow_field_list):
            pos1 = (y - i / tintv_obs * oflow_field[1], 
                    x - i / tintv_obs * oflow_field[0])
            pos2 = (y + (tintv_obs - i) / tintv_obs * oflow_field[1], 
                    x + (tintv_obs - i) / tintv_obs * oflow_field[0])

            fieldt1 = map_coordinates(field_da[0], pos1, order=1)
            fieldt2 = map_coordinates(field_da[1], pos2, order=1)

            field_interp = field_da.isel(time=[0]).copy()
            field_interp[:] = ((tintv_obs - i) * fieldt1 + i * fieldt2) / tintv_obs
            try:
                field_interp.coords['time'] = field_interp['time'] + np.timedelta64(int(new_time - tbgn), 's')
            except TypeError:
                field_interp.coords['time'] = field_interp['time'] + timedelta(seconds=int(new_time - tbgn))
            field_interp.coords['time_seconds'] = new_time
            field_interp_list.append(field_interp)

        radar_ds_interp = xr.merge(field_interp_list)
        radar_ds_list.append(radar_ds_interp)
        
    return radar_ds_list


def advection_correction_ds_old(radar_ds, tintv_obs, tintv, base_field_name='reflectivity_masked', 
                                use_base_field=False, method="LK"):
    # Evaluate advection
    oflow_method = motion.get_method(method)
    fd_kwargs = {"buffer_mask": 10}  # avoid edge effects

    base_field = radar_ds[base_field_name]
    if use_base_field:
        oflow_field = oflow_method(base_field, fd_kwargs=fd_kwargs)
    
    # Perform temporal interpolation on all variables in Dataset using the flow field derived from either 
    # the "base" field (by default, reflectivity), or each field individually
    
    tbgn = base_field[0].coords['time_seconds'].values.item()   # Need to get scalar value, not 0-d
                                                                # numpy array
    print(tbgn)
    print(tintv)
    radar_ds_list = []
    x, y = np.meshgrid(
        np.arange(base_field[0].shape[1], dtype=float), np.arange(base_field[0].shape[0], dtype=float),
    )
    
    new_time = tbgn
    for i in np.arange(tintv, tintv_obs + tintv, tintv):
        new_time = new_time + tintv
        if use_base_field:
            pos1 = (y - i / tintv_obs * oflow_field[1], 
                    x - i / tintv_obs * oflow_field[0])
            pos2 = (y + (tintv_obs - i) / tintv_obs * oflow_field[1], 
                    x + (tintv_obs - i) / tintv_obs * oflow_field[0])
        
        field_interp_list = []
        for field_name, field_da in radar_ds.items():
            if not use_base_field:
                oflow_field = oflow_method(field_da, fd_kwargs=fd_kwargs)
                pos1 = (y - i / tintv_obs * oflow_field[1], 
                        x - i / tintv_obs * oflow_field[0])
                pos2 = (y + (tintv_obs - i) / tintv_obs * oflow_field[1], 
                        x + (tintv_obs - i) / tintv_obs * oflow_field[0])
                
            fieldt1 = map_coordinates(field_da[0], pos1, order=1)
            fieldt2 = map_coordinates(field_da[1], pos2, order=1)
       
            field_interp = field_da.isel(time=[0]).copy()
            field_interp[:] = ((tintv_obs - i) * fieldt1 + i * fieldt2) / tintv_obs
            try:
                field_interp.coords['time'] = field_interp['time'] + np.timedelta64(int(new_time - tbgn), 's')
            except TypeError:
                field_interp.coords['time'] = field_interp['time'] + timedelta(seconds=int(new_time - tbgn))
            field_interp.coords['time_seconds'] = new_time
            field_interp_list.append(field_interp)
        
        radar_ds_interp = xr.merge(field_interp_list)
        radar_ds_list.append(radar_ds_interp)
        
    return radar_ds_list


def advection_correction(arr, tintv_obs, tintv):
    """
    R = np.array([qpe_previous, qpe_current])
    T = time between two observations (5 min)
    t = interpolation timestep (1 min)
    """

    # Evaluate advection
    oflow_method = motion.get_method("LK")
    fd_kwargs = {"buffer_mask": 10}  # avoid edge effects
    V = oflow_method(arr, fd_kwargs=fd_kwargs)

    # Perform temporal interpolation
    # arr_d = np.zeros((arr[0].shape))
    arr_list = []
    x, y = np.meshgrid(
        np.arange(arr[0].shape[1], dtype=float), np.arange(arr[0].shape[0], dtype=float),
    )
    for i in np.arange(tintv, tintv_obs + tintv, tintv):

        pos1 = (y - i / tintv_obs * V[1], x - i / tintv_obs * V[0])
        R1 = map_coordinates(arr[0], pos1, order=1)
        
        pos2 = (y + (tintv_obs - i) / tintv_obs * V[1], x + (tintv_obs - i) / tintv_obs * V[0])
        R2 = map_coordinates(arr[1], pos2, order=1)

        arr_interp = ((tintv_obs - i) * R1 + i * R2) / tintv_obs
        arr_list.append(arr_interp)

    return arr_list


def interp_pyart_grid_to_PIPS(grid_ds, PIPS_loc_list):
    """Interpolate pyart gridded radar to PIPS locations given an xarray dataset with pyart grid info and a list
       of PIPS lats, lons, and altitudes. Returns lists of PIPS x, y, and z locations in radar grid coordinates
       and a list of xarray Datasets for each grid variable interpolated to each PIPS location."""

    # TODO: instead of returning lists, maybe make the PIPS name a dimension of a new xarray Dataset
    try:
        ctrlat = float(grid_ds.origin_latitude)
    except TypeError:
        ctrlat = float(grid_ds.origin_latitude[0])
    try:
        ctrlon = float(grid_ds.origin_longitude)
    except TypeError:
        ctrlon = float(grid_ds.origin_longitude[0])
    print(ctrlat, ctrlon)

    radar_at_PIPS_list = []
    PIPS_x_list = []
    PIPS_y_list = []
    PIPS_z_list = []

    for PIPS_loc in PIPS_locs:
        PIPS_lat = PIPS_loc[0]
        PIPS_lon = PIPS_loc[1]
        PIPS_alt = PIPS_loc[2]
        try:
            radar_alt = float(grid_ds.origin_altitude)
        except TypeError:
            radar_alt = float(grid_ds.origin_altitude[0])
        
        PIPS_z = PIPS_alt - radar_alt
        PIPS_z_list.append(PIPS_z)
        # Use this function to get the x and y coords of the PIPS. Note that this will only be correct 
        # if the radar
        # grid was created using the default pyart aeqd projection.
        PIPS_x, PIPS_y = pyart.core.geographic_to_cartesian_aeqd(PIPS_lon, PIPS_lat, ctrlon, ctrlat)
        PIPS_x = PIPS_x.squeeze().item()
        PIPS_y = PIPS_y.squeeze().item()
        PIPS_x_list.append(PIPS_x)
        PIPS_y_list.append(PIPS_y)
        print('PIPS lat, lon, alt: ', PIPS_lat, PIPS_lon, PIPS_alt)
        print('PIPS x, y, z: ', PIPS_x, PIPS_y, PIPS_z)
        radar_at_PIPS_ds = grid_ds.interp(x=PIPS_x, y=PIPS_y)
        radar_at_PIPS_list.append(radar_at_PIPS_ds)

    return PIPS_x_list, PIPS_y_list, PIPS_z_list, radar_at_PIPS_list

In [ ]:
# Read in the gridded radar data (must have run the combine_gridded_radar notebook first!)
radar_name = 'SMARTR2'

# For 03/30/22 case (PERiLS-2022 IOP2)
date = '0330'
radar_start_datetimestamp = '20220330220251'
radar_end_datetimestamp = '20220331022002'
height = 500.

# Create datetime objects for start and end times
datetime_start = datetime.strptime(radar_start_datetimestamp, '%Y%m%d%H%M%S')
datetime_end = datetime.strptime(radar_end_datetimestamp, '%Y%m%d%H%M%S')

# radar_basedir = \
#     '/Users/dawson29/Projects/PERiLS/obsdata/2022/NEXRAD/GRID/IOP2/KGWX'

radar_basedir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/PERiLS/obsdata/2022/COMMON_GRID/IOP2/SMARTR2'
#radar_basedir = os.path.join(radar_basedir, '{}/{}'.format(date, radar_name[1:]))
gridded_radar_input_dir = radar_basedir
gridded_radar_output_dir = os.path.join(radar_basedir, 'combined')
if not os.path.exists(gridded_radar_output_dir):
    os.makedirs(gridded_radar_output_dir)

radar_start_timestamp = datetime_start.strftime('%H%M%S')
radar_end_timestamp = datetime_end.strftime('%H%M%S')
gridded_radar_combined_filename = '{}_{}_{}_gridded.nc'.format(radar_name, radar_start_timestamp,
                                                               radar_end_timestamp)

gridded_radar_combined_filepath = os.path.join(gridded_radar_output_dir, gridded_radar_combined_filename)
gridded_radar_xr = xr.open_dataset(gridded_radar_combined_filepath)
print(gridded_radar_xr)

In [ ]:
plot_dir = os.path.join(gridded_radar_output_dir, 'plots')
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

In [ ]:
# Read in PIPS data
deployment = 'IOP2_033022'
PIPS_list = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS3B']
PIPS_data_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data/IOP2_033022/netcdf_60s'

PIPS_ds_list = []
PIPS_locs = []

for PIPS in PIPS_list:
    PIPS_filename = 'parsivel_combined_{}_{}_60s.nc'.format(deployment, PIPS)
    PIPS_filepath = os.path.join(PIPS_data_dir, PIPS_filename)
    PIPS_ds = xr.load_dataset(PIPS_filepath)
    PIPS_ds_list.append(PIPS_ds)
    PIPS_loc = eval(PIPS_ds.location)
    PIPS_locs.append(PIPS_loc)

In [ ]:
PIPS_x_list, PIPS_y_list, PIPS_z_list, radar_at_PIPS_ds_list = interp_pyart_grid_to_PIPS(gridded_radar_xr, 
                                                                                         PIPS_locs)
    
PIPS_z_mean = np.array(PIPS_z_list).mean()
print(PIPS_z_list)
print(PIPS_z_mean)
PIPS_x_arr = np.array(PIPS_x_list)
PIPS_y_arr = np.array(PIPS_y_list)

print(PIPS_x_arr, PIPS_y_arr)

In [ ]:
datetime_substart = datetime_start
datetime_subend = datetime_end
timestamp_substart = datetime_substart.strftime('%Y-%m-%dT%H:%M')
timestamp_subend = datetime_subend.strftime('%Y-%m-%dT%H:%M')

gridded_radar_xr_subset = gridded_radar_xr.sel(time=slice(timestamp_substart, timestamp_subend))
print(gridded_radar_xr_subset['time'])

# Round times to nearest minute
rounded_times = gridded_radar_xr_subset['time'].dt.round('60S')
print(rounded_times)
gridded_radar_xr_subset['time'] = rounded_times

time_seconds = (gridded_radar_xr_subset['time'] - gridded_radar_xr_subset['time'][0]) / np.timedelta64(1, 's')
print(time_seconds)
gridded_radar_xr_subset.coords['time_seconds'] = ('time', time_seconds.data)
print(gridded_radar_xr_subset)
# total_seconds = (gridded_radar_xr_subset['time'] - gridded_radar_xr_subset['time'][0]).total_seconds()

In [ ]:
# Extract one level from the gridded radar Dataset
gridded_radar_xr_subset_onelevel = gridded_radar_xr_subset.sel(z=height).squeeze()
# gridded_radar_xr_subset_onelevel = gridded_radar_xr_subset.interp(z=height).squeeze()
gridded_radar_xr_subset_onelevel = gridded_radar_xr_subset_onelevel.transpose("time", "y", "x")

In [ ]:
print(gridded_radar_xr_subset_onelevel)

In [ ]:
ref_varname = 'REF'
zdr_varname = 'ZDR'
rhv_varname = 'RHO'

In [ ]:
oflow_method = motion.get_method("LK")
fd_kwargs = {"buffer_mask": 10} # avoid edge effects

# Plot optical flow field for reflectivity as a function of time

clevels = np.arange(0., 61., 1.)
norm = cm.colors.Normalize(vmin=clevels[0], vmax=clevels[-1])
cmap = 'pyart_HomeyerRainbow'
# cmap = cm.plasma
cbarintv = 5.
# cbarlevels = ticker.MultipleLocator(base=cbarintv)
cbarlevels = np.arange(clevels[0], clevels[-1] + cbarintv, cbarintv)
xplt = gridded_radar_xr_subset_onelevel['x'].values
yplt = gridded_radar_xr_subset_onelevel['y'].values
dBZ = gridded_radar_xr_subset_onelevel[ref_varname]
times = dBZ['time'][1:-1]
print(times)

uplt_mean_list = []
vplt_mean_list = []
for t, curtime in enumerate(times):
    # print(t, t+2)
    # print(curtime)
    curtime_datetime = pd.to_datetime(curtime.values).to_pydatetime()
    # TODO: the following works (and the above no longer does). Need to find out what is really
    # going on with all these different datetime classes
    # EDIT: ok, now this no longer works, so reverting back to the previous. This is insane...
    # curtime_datetime = curtime.values.item()
    print(curtime_datetime)
    curtime_stamp = curtime_datetime.strftime('%Y%m%d%H%M%S')
    velocity_field = oflow_method(dBZ[t:t+2], fd_kwargs=fd_kwargs)
    qintv = 10
    uplt = velocity_field[0]
    vplt = velocity_field[1]
    fig, ax = plt.subplots(figsize=(8, 8))
    dBZ_plt = ax.contourf(xplt, yplt, dBZ[t+1].values, levels=clevels, cmap=cmap, norm=norm)
    ax.quiver(xplt[::qintv], yplt[::qintv], uplt[::qintv,::qintv], vplt[::qintv,::qintv])
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(dBZ_plt, orientation='vertical', cax=cax, ticks=cbarlevels)
    cax.set_ylabel('dBZ')
    formatter = ticker.FuncFormatter(mtokm)
    ax.xaxis.set_major_formatter(formatter)
    ax.yaxis.set_major_formatter(formatter)
    # ax.xaxis.set_major_locator(ticker.MultipleLocator(base=axestickintv))
    # ax.yaxis.set_major_locator(ticker.MultipleLocator(base=axestickintv))
    ax.set_xlabel('km')
    ax.set_ylabel('km')
    ax.set_aspect('equal')

    plot_filename='pysteps_adv_correction_{}.png'.format(curtime_stamp)
    plot_filepath = os.path.join(plot_dir, plot_filename)
    fig.savefig(plot_filepath, dpi=300, bbox_inches='tight')

    print(uplt.mean(), vplt.mean())
    uplt_mean_list.append(uplt.mean())
    vplt_mean_list.append(vplt.mean())

In [ ]:
# Perform advection correction and interpolate gridded radar sequence to 1-min intervals
# Just do Z, ZDR, and RHV

# gridded_Z_ZDR_RHV = gridded_radar_xr_subset_onelevel[['reflectivity', 
#                                                       'differential_reflectivity',
#                                                       'cross_correlation_ratio']]

gridded_Z_ZDR_RHV = gridded_radar_xr_subset_onelevel[[ref_varname, zdr_varname, rhv_varname]]
tinterp_intv = 60.

print(gridded_Z_ZDR_RHV)

# print(gridded_Z_ZDR.isel(time=[0]))

# for varname, var in gridded_Z_ZDR.items():
#     print(varname)
#     print(var[1])

gridded_Z_ZDR_RHV_interp_list = interpolate_all(gridded_Z_ZDR_RHV, tinterp_intv, 
                                                base_field_name=ref_varname, use_base_field=True)

# gridded_Z_ZDR_RHV_interp_list = interpolate_all(gridded_Z_ZDR_RHV, tinterp_intv, 
#                                                 base_field_name=ref_varname, use_base_field=True)


In [ ]:
gridded_Z_ZDR_RHV_interp_ds = xr.concat(gridded_Z_ZDR_RHV_interp_list, 'time')
gridded_Z_ZDR_RHV_interp_ds

In [ ]:
# Check to see if masked versions of ZH, ZDR, and RHV exist in the file. 
# If not, create them here.

mask_anyway = True
ZH_thresh = 5.
ZDR_thresh = 0.1
RHV_thresh = 0.95

print("Z, ZDR, RHV thresholds are {:f}, {:f}, {:f}".format(ZH_thresh, ZDR_thresh, RHV_thresh))

if 'reflectivity_masked' not in gridded_Z_ZDR_RHV_interp_ds or mask_anyway:
    print("Masking Z, ZDR, and RHV on thresholds of {:f}, {:f}, {:f}".format(ZH_thresh, ZDR_thresh, RHV_thresh))
    ZH_mask = np.where(gridded_Z_ZDR_RHV_interp_ds[ref_varname] < ZH_thresh, True, False)
    ZH_mask = np.where(np.isfinite(gridded_Z_ZDR_RHV_interp_ds[ref_varname]), ZH_mask, False)
    ZDR_mask = np.where(gridded_Z_ZDR_RHV_interp_ds[zdr_varname] < ZDR_thresh, True, False)
    ZDR_mask = np.where(np.isfinite(gridded_Z_ZDR_RHV_interp_ds[zdr_varname]), ZDR_mask, False)
    RHV_mask = np.where(gridded_Z_ZDR_RHV_interp_ds[rhv_varname] < RHV_thresh, True, False)
    RHV_mask = np.where(np.isfinite(gridded_Z_ZDR_RHV_interp_ds[rhv_varname]), RHV_mask, False)
    full_mask = np.ma.mask_or(ZH_mask, ZDR_mask)
    full_mask = np.ma.mask_or(full_mask, RHV_mask)

    gridded_Z_ZDR_RHV_interp_ds[f'{ref_varname}_masked'] = \
        gridded_Z_ZDR_RHV_interp_ds[ref_varname].where(~full_mask)
    gridded_Z_ZDR_RHV_interp_ds[f'{zdr_varname}_masked'] = \
        gridded_Z_ZDR_RHV_interp_ds[zdr_varname].where(~full_mask)
    gridded_Z_ZDR_RHV_interp_ds[f'{rhv_varname}_masked'] = \
        gridded_Z_ZDR_RHV_interp_ds[rhv_varname].where(~full_mask)
    print("Writing {}".format(gridded_radar_combined_filename))
    #gridded_radar_xr.to_netcdf(gridded_radar_combined_filepath)
print(gridded_Z_ZDR_RHV_interp_ds)

In [ ]:
# Save interpolated grid to file
out_start_timestamp = datetime_substart.strftime('%Y%m%d%H%M')
out_end_timestamp = datetime_subend.strftime('%Y%m%d%H%M')
gridded_radar_interp_filename = '{}_{}_{}_z{:d}_gridded_interp.nc'.format(radar_name, out_start_timestamp,
                                                                          out_end_timestamp, int(height))
gridded_radar_interp_filepath = os.path.join(gridded_radar_output_dir, gridded_radar_interp_filename)
gridded_Z_ZDR_RHV_interp_ds.to_netcdf(gridded_radar_interp_filepath)

In [ ]:
# Plot reflectivity for time-interpolated grid
# Choose a subset of times to keep animation size down
anim_start = '2022-03-30T23:30'
anim_end = '2022-03-31T01:00'

var_da = gridded_Z_ZDR_RHV_interp_ds[f'{ref_varname}'].sel(time=slice(anim_start, anim_end))
xplt = var_da.coords['x']
yplt = var_da.coords['y']

clevels =np.arange(0., 61., 1.)
norm = cm.colors.Normalize(vmin=0., vmax=60.)

import matplotlib.animation as animation
fig, ax = plt.subplots(figsize=(8, 8))

ims = []
for i, var in enumerate(var_da):
    time = var.coords['time_seconds'].values.item()
    ci = ax.contourf(xplt, yplt, var.squeeze(), levels=clevels, 
                     cmap='pyart_HomeyerRainbow', norm=norm)
    # Plot PIPS location
    #ax.plot([PIPS_x], [PIPS_y], 'k*')
    if i == 0.:
        fig.colorbar(ci, ax=ax)
        ax.set_aspect('equal')
    ims.append(ci.collections)
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000)
plt.close()
HTML(ani.to_jshtml())

In [ ]:
# Save above animation to disk
ani_filename = 'interp_ref_{}_{}_z{:d}.mp4'.format(anim_start, anim_end, int(height))
ani_filepath = os.path.join(plot_dir, ani_filename)

ani.save(ani_filepath)

In [ ]:
# Plot ZDR for time-interpolated grid
# Choose a subset of times to keep animation size down
anim_start = '2022-03-30T23:30'
anim_end = '2022-03-31T01:00'

var_da = gridded_Z_ZDR_RHV_interp_ds[f'{zdr_varname}'].sel(time=slice(anim_start, anim_end))
xplt = var_da.coords['x']
yplt = var_da.coords['y']

clevels =np.arange(0., 6.1, 0.1)
norm = cm.colors.Normalize(vmin=0., vmax=6.)

import matplotlib.animation as animation
fig, ax = plt.subplots(figsize=(8, 8))

ims = []
for i, var in enumerate(var_da):
    time = var.coords['time_seconds'].values.item()
    ci = ax.contourf(xplt, yplt, var.squeeze(), levels=clevels, 
                     cmap='pyart_HomeyerRainbow', norm=norm)
    # Plot PIPS location
    #ax.plot([PIPS_x], [PIPS_y], 'k*')
    if i == 0.:
        fig.colorbar(ci, ax=ax)
        ax.set_aspect('equal')
    ims.append(ci.collections)
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000)
plt.close()
HTML(ani.to_jshtml())

In [ ]:
# Save above animation to disk
ani_filename = 'interp_zdr_{}_{}_z{:d}.mp4'.format(anim_start, anim_end, int(height))
ani_filepath = os.path.join(plot_dir, ani_filename)

ani.save(ani_filepath)

In [ ]:
gridded_radar_xr_subset_onelevel